In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle -q
!pip install catboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 25.8 MB/s eta 0:00:00


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mertbayraktar","key":"1c4f5e98b285e71de3134437dc08c463"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c brist1d

 87% 8.00M/9.21M [00:01<00:00, 12.8MB/s]
100% 9.21M/9.21M [00:01<00:00, 8.67MB/s]


In [ ]:
!mkdir -p ./data/brist-1d
!unzip brist1d.zip -d ./data/brist-1d
!ls ./data/brist-1d

Archive:  brist1d.zip
  inflating: ./data/brist-1d/activities.txt  
  inflating: ./data/brist-1d/sample_submission.csv  
  inflating: ./data/brist-1d/test.csv  
  inflating: ./data/brist-1d/train.csv  
activities.txt	sample_submission.csv  test.csv  train.csv


In [ ]:
import numpy as np
import pandas as pd


import lightgbm as lgb
import catboost as cb
import xgboost as xgb

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GroupKFold

from math import sqrt

import shutil


import os
import joblib

import scipy.stats as stats



pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option("display.max_rows", None)

In [ ]:
train = pd.read_csv('/content/data/brist-1d/train.csv')
train.head()


<ipython-input-210-a6ddce625ffe>:1: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('/content/data/brist-1d/train.csv')


,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,bg-5:15,bg-5:10,bg-5:05,bg-5:00,bg-4:55,bg-4:50,bg-4:45,bg-4:40,bg-4:35,bg-4:30,bg-4:25,bg-4:20,bg-4:15,bg-4:10,bg-4:05,bg-4:00,bg-3:55,bg-3:50,bg-3:45,bg-3:40,bg-3:35,bg-3:30,bg-3:25,bg-3:20,bg-3:15,bg-3:10,bg-3:05,bg-3:00,bg-2:55,bg-2:50,bg-2:45,bg-2:40,bg-2:35,bg-2:30,bg-2:25,bg-2:20,bg-2:15,bg-2:10,bg-2:05,bg-2:00,bg-1:55,bg-1:50,bg-1:45,bg-1:40,bg-1:35,bg-1:30,bg-1:25,bg-1:20,bg-1:15,bg-1:10,bg-1:05,bg-1:00,bg-0:55,bg-0:50,bg-0:45,bg-0:40,bg-0:35,bg-0:30,bg-0:25,bg-0:20,bg-0:15,bg-0:10,bg-0:05,bg-0:00,insulin-5:55,insulin-5:50,insulin-5:45,insulin-5:40,insulin-5:35,insulin-5:30,insulin-5:25,insulin-5:20,insulin-5:15,insulin-5:10,insulin-5:05,insulin-5:00,insulin-4:55,insulin-4:50,insulin-4:45,insulin-4:40,insulin-4:35,insulin-4:30,insulin-4:25,insulin-4:20,insulin-4:15,insulin-4:10,insulin-4:05,insulin-4:00,insulin-3:55,insulin-3:50,insulin-3:45,insulin-3:40,insulin-3:35,insulin-3:30,insulin-3:25,insulin-3:20,insulin-3:15,insulin-3:10,insulin-3:05,insulin-3:00,insulin-2:55,insulin-2:50,insulin-2:45,insulin-2:40,insulin-2:35,insulin-2:30,insulin-2:25,insulin-2:20,insulin-2:15,insulin-2:10,insulin-2:05,insulin-2:00,insulin-1:55,insulin-1:50,insulin-1:45,insulin-1:40,insulin-1:35,insulin-1:30,insulin-1:25,insulin-1:20,insulin-1:15,insulin-1:10,insulin-1:05,insulin-1:00,insulin-0:55,insulin-0:50,insulin-0:45,insulin-0:40,insulin-0:35,insulin-0:30,insulin-0:25,insulin-0:20,insulin-0:15,insulin-0:10,insulin-0:05,insulin-0:00,carbs-5:55,carbs-5:50,carbs-5:45,carbs-5:40,carbs-5:35,carbs-5:30,carbs-5:25,carbs-5:20,carbs-5:15,carbs-5:10,carbs-5:05,carbs-5:00,carbs-4:55,carbs-4:50,carbs-4:45,carbs-4:40,carbs-4:35,carbs-4:30,carbs-4:25,carbs-4:20,carbs-4:15,carbs-4:10,carbs-4:05,carbs-4:00,carbs-3:55,carbs-3:50,carbs-3:45,carbs-3:40,carbs-3:35,carbs-3:30,carbs-3:25,carbs-3:20,carbs-3:15,carbs-3:10,carbs-3:05,carbs-3:00,carbs-2:55,carbs-2:50,carbs-2:45,carbs-2:40,carbs-2:35,carbs-2:30,carbs-2:25,carbs-2:20,carbs-2:15,carbs-2:10,carbs-2:05,carbs-2:00,carbs-1:55,carbs-1:50,carbs-1:45,carbs-1:40,carbs-1:35,carbs-1:30,carbs-1:25,carbs-1:20,carbs-1:15,carbs-1:10,carbs-1:05,carbs-1:00,carbs-0:55,carbs-0:50,carbs-0:45,carbs-0:40,carbs-0:35,carbs-0:30,carbs-0:25,carbs-0:20,carbs-0:15,carbs-0:10,carbs-0:05,carbs-0:00,hr-5:55,hr-5:50,hr-5:45,hr-5:40,hr-5:35,hr-5:30,hr-5:25,hr-5:20,hr-5:15,hr-5:10,hr-5:05,hr-5:00,hr-4:55,hr-4:50,hr-4:45,hr-4:40,hr-4:35,hr-4:30,hr-4:25,hr-4:20,hr-4:15,hr-4:10,hr-4:05,hr-4:00,hr-3:55,hr-3:50,hr-3:45,hr-3:40,hr-3:35,hr-3:30,hr-3:25,hr-3:20,hr-3:15,hr-3:10,hr-3:05,hr-3:00,hr-2:55,hr-2:50,hr-2:45,hr-2:40,hr-2:35,hr-2:30,hr-2:25,hr-2:20,hr-2:15,hr-2:10,hr-2:05,hr-2:00,hr-1:55,hr-1:50,hr-1:45,hr-1:40,hr-1:35,hr-1:30,hr-1:25,hr-1:20,hr-1:15,hr-1:10,hr-1:05,hr-1:00,hr-0:55,hr-0:50,hr-0:45,hr-0:40,hr-0:35,hr-0:30,hr-0:25,hr-0:20,hr-0:15,hr-0:10,hr-0:05,hr-0:00,steps-5:55,steps-5:50,steps-5:45,steps-5:40,steps-5:35,steps-5:30,steps-5:25,steps-5:20,steps-5:15,steps-5:10,steps-5:05,steps-5:00,steps-4:55,steps-4:50,steps-4:45,steps-4:40,steps-4:35,steps-4:30,steps-4:25,steps-4:20,steps-4:15,steps-4:10,steps-4:05,steps-4:00,steps-3:55,steps-3:50,steps-3:45,steps-3:40,steps-3:35,steps-3:30,steps-3:25,steps-3:20,steps-3:15,steps-3:10,steps-3:05,steps-3:00,steps-2:55,steps-2:50,steps-2:45,steps-2:40,steps-2:35,steps-2:30,steps-2:25,steps-2:20,steps-2:15,steps-2:10,steps-2:05,steps-2:00,steps-1:55,steps-1:50,steps-1:45,steps-1:40,steps-1:35,steps-1:30,steps-1:25,steps-1:20,steps-1:15,steps-1:10,steps-1:05,steps-1:00,steps-0:55,steps-0:50,steps-0:45,steps-0:40,steps-0:35,steps-0:30,steps-0:25,steps-0:20,steps-0:15,steps-0:10,steps-0:05,steps-0:00,cals-5:55,cals-5:50,cals-5:45,cals-5:40,cals-5:35,cals-5:30,cals-5:25,cals-5:20,cals-5:15,cals-5:10,cals-5:05,cals-5:00,cals-4:55,cals-4:50,cals-4:45,cals-4:40,cals-4:35,cals-4:30,cals-4:25,cals-4:20,cals-4:15,cals-4:10,cals-4:05,cals-4:00,cals-3:55,cals-3:50,cals-3:45,cals-3:40,cals-3:35,cals-3:30,cals-3:25,cal

In [ ]:
class CONFIG:

    TARGET_COL = 'bg+1+00'
    GROUP_COL = 'p_num'
    N_SPLITS = 5
    # SEED_LIST = [2,12,22,32,42,52,62,72,82,92,102,112,122,132,142,152]
    SEED_LIST = [42]

    ERR = 1e-5

    BG_LOW = 3.9
    BG_HIGH = 10.0


    TRAIN_CATBOOST = False
    TRAIN_XGB = True
    TRAIN_LGB = False

    early_stop = 200

    catboost_params = {
        # 'random_state': 42,  # This will be updated per seed
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'learning_rate': 0.025,
        'iterations': 1000,
        'task_type': 'CPU',
        'depth': 7,
    }

    xgboost_params = {
        'random_state': 42,  # This will be updated per seed
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'learning_rate': 0.025,
        'tree_method': 'hist',
        'device': 'cpu',
        'max_depth': 7,
    }


    lgb_params = {
        # 'random_state': 42,  # This will be updated per seed
        'objective': 'regression',
        'metric': 'rmse',
        'boosting': 'gbdt',
        'num_threads': 8,
        'learning_rate': 0.025,
        # 'n_estimators': 20000,
        'device': 'cpu',
        'max_depth': 5,
        'num_leaves': 64,
        # 'min_data_in_leaf': 50,
        # 'lambda_l1': 30,
        # 'lambda_l2': 10,
        'verbosity': -1
    }

In [ ]:
def _extract_time(col: str) -> float:
    time_str = col.split('-')[-1] if '-' in col else col
    hours, minutes = map(int, time_str.split('+'))
    return hours + minutes/60

def _within_window(col: str, window: str) -> bool:
    try:
        col_time = _extract_time(col)
        window_time = _extract_time(window)
        return col_time <= window_time
    except:
        return False

In [ ]:
# @title feature engineering

def calculate_patient_features(df: pd.DataFrame) -> pd.DataFrame:

    bg_columns = [col for col in df.columns if col.startswith('bg-')]
    insulin_columns = [col for col in df.columns if col.startswith('insulin-')]
    hr_columns = [col for col in df.columns if col.startswith('hr-')]
    steps_columns = [col for col in df.columns if col.startswith('steps-')]
    cals_columns = [col for col in df.columns if col.startswith('cals-')]


    df['bg_sum_5hr'] = df[bg_columns].sum(axis=1)
    df['bg_mean_5hr'] = df['bg_sum_5hr'] / len(bg_columns)

    df['insulin_sum_5hr'] = df[insulin_columns].sum(axis=1)
    df['insulin_mean_5hr'] = df['insulin_sum_5hr'] / len(insulin_columns)

    df['hr_sum_5hr'] = df[hr_columns].sum(axis=1)
    df['hr_mean_5hr'] = df['hr_sum_5hr'] / len(hr_columns)

    df['steps_sum_5hr'] = df[steps_columns].sum(axis=1)
    df['steps_mean_5hr'] = df['steps_sum_5hr'] / len(steps_columns)

    df['cals_sum_5hr'] = df[cals_columns].sum(axis=1)
    df['cals_mean_5hr'] = df['cals_sum_5hr'] / len(cals_columns)

    return df

In [ ]:
def preprocess(df):

  df.columns = df.columns.str.replace(':', '+', regex=False)

  df['time'] = pd.to_datetime(df['time'])
  df['hour'] = df['time'].dt.hour
  df['minute'] = df['time'].dt.minute

  df['part_of_day'] = pd.cut(df['hour'],
                                 bins=[-np.inf, 6, 12, 18, np.inf],
                                 labels=['night', 'morning', 'afternoon', 'evening'])

  df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
  df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)

  cat_cols = df.select_dtypes(include='object').columns.tolist()
  numeric_cols = df.select_dtypes(include=['number']).columns.tolist()

  df[cat_cols] = df[cat_cols].fillna('Unknown').astype('category')
  df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill')

  df = calculate_patient_features(df)

  df.drop(df.filter(regex='carbs|activity').columns, axis=1, inplace=True)

  df.drop(['time'], axis=1, inplace=True)
  df.drop(['id'], axis=1, inplace=True)

  return df

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177024 entries, 0 to 177023
Columns: 508 entries, id to bg+1:00
dtypes: float64(433), object(75)
memory usage: 686.1+ MB


In [ ]:
train.isnull().sum()

,0
id,0
p_num,0
time,0
bg-5:55,27254
bg-5:50,18491
bg-5:45,13660
bg-5:40,27258
bg-5:35,18770
bg-5:30,13254
bg-5:25,27261


In [ ]:
train['p_num'].value_counts()

,count
p_num,
p03,26028
p02,25872
p10,25454
p12,25299
p04,24686
p11,24555
p01,8459
p06,8383
p05,8288


In [ ]:
# @title train_catboost
def train_catboost(data, model_save_dir='catboost_models'):

    if os.path.exists(model_save_dir):
        shutil.rmtree(model_save_dir)

    # Create the directory
    os.makedirs(model_save_dir)

    X = data.drop([CONFIG.TARGET_COL, CONFIG.GROUP_COL], axis=1)
    y = data[CONFIG.TARGET_COL]
    groups = data[CONFIG.GROUP_COL]

    cv = GroupKFold(n_splits=CONFIG.N_SPLITS)
    all_oof_preds = np.zeros(X.shape[0])
    all_metrics = []
    all_models = []
    feature_importances = np.zeros(X.shape[1])

    seed_oof_preds = []

    cat_features = [X.columns.get_loc(col) for col in X.select_dtypes(include=['category'])]

    for seed in CONFIG.SEED_LIST:
        print(f'Training with seed: {seed} for feature importance calculation...')
        CONFIG.catboost_params['random_state'] = seed

        oof_preds = np.zeros(X.shape[0])
        metrics = []
        models = []

        for fi, (train_idx, valid_idx) in enumerate(cv.split(X, y, groups)):

            print(f'Fold {fi + 1}/{CONFIG.N_SPLITS} with seed {seed}...')

            model = cb.CatBoostRegressor(**CONFIG.catboost_params)

            model.fit(X.iloc[train_idx], y.iloc[train_idx],
                      eval_set=(X.iloc[valid_idx], y.iloc[valid_idx]),
                      use_best_model=True,
                      # early_stopping_rounds=Config.early_stop,
                      cat_features=cat_features,
                      verbose=200)

            preds = model.predict(X.iloc[valid_idx])
            oof_preds[valid_idx] = preds

            rmse = sqrt(mean_squared_error(y.iloc[valid_idx], preds))
            metrics.append(rmse)
            print(f'Fold {fi + 1} RMSE: {rmse:.4f}')

            models.append(model)

            model_save_path = os.path.join(model_save_dir, f'catboost_model_seed_{seed}_fold_{fi}.pkl')
            joblib.dump(model, model_save_path)

            feature_importances += model.get_feature_importance()

        seed_oof_preds.append(oof_preds)
        all_models.append(models)
        all_metrics.append(metrics)

    oof_preds_avg = np.mean(seed_oof_preds, axis=0)
    oof_rmse = sqrt(mean_squared_error(y, oof_preds_avg))

    all_fold_rmses = np.array([rmse for metrics in all_metrics for rmse in metrics])

    print(f'Average RMSE across seeds: {np.mean([np.mean(m) for m in all_metrics]):.4f}')
    print(f'STD of RMSE across folds: {np.std(all_fold_rmses):.4f}')
    print(f'OOF RMSE across seeds: {oof_rmse:.4f}')

        # t-statistic and p-value calculation
    seed_fold_rmses = np.array([[rmse for rmse in metrics] for metrics in all_metrics])

    for i in range(len(CONFIG.SEED_LIST)):
        for j in range(i + 1, len(CONFIG.SEED_LIST)):
            t_stat, p_value = stats.ttest_rel(seed_fold_rmses[i], seed_fold_rmses[j])
            print(f'T-statistic between seed {CONFIG.SEED_LIST[i]} and {CONFIG.SEED_LIST[j]}: {t_stat:.4f}, p-value: {p_value:.4f}')

    return all_models


In [ ]:
# @title train_xgboost
def train_xgboost(data):

    X = data.drop([CONFIG.TARGET_COL, CONFIG.GROUP_COL], axis=1)
    y = data[CONFIG.TARGET_COL]
    groups = data[CONFIG.GROUP_COL]

    cv = GroupKFold(n_splits=CONFIG.N_SPLITS)
    all_oof_preds = np.zeros(X.shape[0])
    all_metrics = []
    all_models = []
    feature_importances = np.zeros(X.shape[1])

    seed_oof_preds = []

    # Identify categorical columns by their dtype (assuming they are still labeled as 'category')
    cat_cols = X.select_dtypes(include='category').columns

    for seed in CONFIG.SEED_LIST:
        print(f'Training with seed: {seed} for feature importance calculation...')
        # CONFIG.xgboost_params['random_state'] = seed

        oof_preds = np.zeros(X.shape[0])
        metrics = []
        models = []

        for fi, (train_idx, valid_idx) in enumerate(cv.split(X, y, groups)):

            # if fi != 0:
            #     continue

            print(f'Fold {fi + 1}/{CONFIG.N_SPLITS} with seed {seed}...')

            dtrain = xgb.DMatrix(X.iloc[train_idx], label=y.iloc[train_idx], enable_categorical=True)
            dvalid = xgb.DMatrix(X.iloc[valid_idx], label=y.iloc[valid_idx], enable_categorical=True)

            model = xgb.train(
                params=CONFIG.xgboost_params,
                dtrain=dtrain,
                num_boost_round=1000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                verbose_eval=200,
                early_stopping_rounds=CONFIG.early_stop,
            )

            preds = model.predict(xgb.DMatrix(X.iloc[valid_idx], enable_categorical=True))
            oof_preds[valid_idx] = preds

            rmse = sqrt(mean_squared_error(y.iloc[valid_idx], preds))
            metrics.append(rmse)
            print(f'Fold {fi + 1} RMSE: {rmse:.4f}')

            models.append(model)

        seed_oof_preds.append(oof_preds)
        all_models.append(models)
        all_metrics.append(metrics)

    oof_preds_avg = np.mean(seed_oof_preds, axis=0)
    oof_rmse = sqrt(mean_squared_error(y, oof_preds_avg))

    all_fold_rmses = np.array([rmse for metrics in all_metrics for rmse in metrics])

    print(f'Average RMSE across seeds: {np.mean([np.mean(m) for m in all_metrics]):.4f}')
    print(f'STD of RMSE across folds: {np.std(all_fold_rmses):.4f}')
    print(f'OOF RMSE across seeds: {oof_rmse:.4f}')

    return all_models

In [ ]:
# @title train_lgb
def train_lgb(data):
    X = data.drop([CONFIG.TARGET_COL, CONFIG.GROUP_COL], axis=1)
    y = data[CONFIG.TARGET_COL]
    groups = data[CONFIG.GROUP_COL]

    cv = GroupKFold(n_splits=CONFIG.N_SPLITS)
    all_oof_preds = np.zeros(X.shape[0])
    all_metrics = []
    all_models = []

    seed_oof_preds = []

    for seed in CONFIG.SEED_LIST:
        print(f'Training with seed: {seed} for feature importance calculation...')

        oof_preds = np.zeros(X.shape[0])
        metrics = []
        models = []

        for fi, (train_idx, valid_idx) in enumerate(cv.split(X, y, groups)):

            print(f'Fold {fi + 1}/{CONFIG.N_SPLITS} with seed {seed}...')
            dtrain = lgb.Dataset(X.iloc[train_idx], label=y.iloc[train_idx])
            dvalid = lgb.Dataset(X.iloc[valid_idx], label=y.iloc[valid_idx], reference=dtrain)

            model = lgb.train(
                params={**CONFIG.lgb_params, 'random_state': seed},
                train_set=dtrain,
                valid_sets=[dtrain, dvalid],
                num_boost_round=1000,
                callbacks=[lgb.early_stopping(stopping_rounds=CONFIG.early_stop), lgb.log_evaluation(period=200)]
            )


            preds = model.predict(X.iloc[valid_idx])
            oof_preds[valid_idx] = preds

            rmse = sqrt(mean_squared_error(y.iloc[valid_idx], preds))
            metrics.append(rmse)
            print(f'Fold {fi + 1} RMSE: {rmse:.4f}')

            models.append(model)

        seed_oof_preds.append(oof_preds)
        all_models.append(models)
        all_metrics.append(metrics)

    oof_preds_avg = np.mean(seed_oof_preds, axis=0)
    oof_rmse = sqrt(mean_squared_error(y, oof_preds_avg))

    all_fold_rmses = np.array([rmse for metrics in all_metrics for rmse in metrics])

    print(f'Average RMSE across seeds: {np.mean([np.mean(m) for m in all_metrics]):.4f}')
    print(f'STD of RMSE across folds: {np.std(all_fold_rmses):.4f}')
    print(f'OOF RMSE across seeds: {oof_rmse:.4f}')


    seed_fold_rmses = np.array([[rmse for rmse in metrics] for metrics in all_metrics])

    for i in range(len(CONFIG.SEED_LIST)):
        for j in range(i + 1, len(CONFIG.SEED_LIST)):
            t_stat, p_value = stats.ttest_rel(seed_fold_rmses[i], seed_fold_rmses[j])
            print(f'T-statistic between seed {CONFIG.SEED_LIST[i]} and {CONFIG.SEED_LIST[j]}: {t_stat:.4f}, p-value: {p_value:.4f}')

    return all_models


In [ ]:
%%time
train = preprocess(train)

train.shape

<ipython-input-214-5f70c4490aba>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'])
<ipython-input-214-5f70c4490aba>:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill')
<ipython-input-213-bc80ef1b55ef>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['bg_sum_5hr'] = df[bg_columns].sum(axis=1)
<ipython-input-213-bc80ef1b55ef>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of

CPU times: user 5.39 s, sys: 113 ms, total: 5.5 s
Wall time: 5.41 s


(177024, 377)

In [ ]:
train.head()

,p_num,bg-5+55,bg-5+50,bg-5+45,bg-5+40,bg-5+35,bg-5+30,bg-5+25,bg-5+20,bg-5+15,bg-5+10,bg-5+05,bg-5+00,bg-4+55,bg-4+50,bg-4+45,bg-4+40,bg-4+35,bg-4+30,bg-4+25,bg-4+20,bg-4+15,bg-4+10,bg-4+05,bg-4+00,bg-3+55,bg-3+50,bg-3+45,bg-3+40,bg-3+35,bg-3+30,bg-3+25,bg-3+20,bg-3+15,bg-3+10,bg-3+05,bg-3+00,bg-2+55,bg-2+50,bg-2+45,bg-2+40,bg-2+35,bg-2+30,bg-2+25,bg-2+20,bg-2+15,bg-2+10,bg-2+05,bg-2+00,bg-1+55,bg-1+50,bg-1+45,bg-1+40,bg-1+35,bg-1+30,bg-1+25,bg-1+20,bg-1+15,bg-1+10,bg-1+05,bg-1+00,bg-0+55,bg-0+50,bg-0+45,bg-0+40,bg-0+35,bg-0+30,bg-0+25,bg-0+20,bg-0+15,bg-0+10,bg-0+05,bg-0+00,insulin-5+55,insulin-5+50,insulin-5+45,insulin-5+40,insulin-5+35,insulin-5+30,insulin-5+25,insulin-5+20,insulin-5+15,insulin-5+10,insulin-5+05,insulin-5+00,insulin-4+55,insulin-4+50,insulin-4+45,insulin-4+40,insulin-4+35,insulin-4+30,insulin-4+25,insulin-4+20,insulin-4+15,insulin-4+10,insulin-4+05,insulin-4+00,insulin-3+55,insulin-3+50,insulin-3+45,insulin-3+40,insulin-3+35,insulin-3+30,insulin-3+25,insulin-3+20,insulin-3+15,insulin-3+10,insulin-3+05,insulin-3+00,insulin-2+55,insulin-2+50,insulin-2+45,insulin-2+40,insulin-2+35,insulin-2+30,insulin-2+25,insulin-2+20,insulin-2+15,insulin-2+10,insulin-2+05,insulin-2+00,insulin-1+55,insulin-1+50,insulin-1+45,insulin-1+40,insulin-1+35,insulin-1+30,insulin-1+25,insulin-1+20,insulin-1+15,insulin-1+10,insulin-1+05,insulin-1+00,insulin-0+55,insulin-0+50,insulin-0+45,insulin-0+40,insulin-0+35,insulin-0+30,insulin-0+25,insulin-0+20,insulin-0+15,insulin-0+10,insulin-0+05,insulin-0+00,hr-5+55,hr-5+50,hr-5+45,hr-5+40,hr-5+35,hr-5+30,hr-5+25,hr-5+20,hr-5+15,hr-5+10,hr-5+05,hr-5+00,hr-4+55,hr-4+50,hr-4+45,hr-4+40,hr-4+35,hr-4+30,hr-4+25,hr-4+20,hr-4+15,hr-4+10,hr-4+05,hr-4+00,hr-3+55,hr-3+50,hr-3+45,hr-3+40,hr-3+35,hr-3+30,hr-3+25,hr-3+20,hr-3+15,hr-3+10,hr-3+05,hr-3+00,hr-2+55,hr-2+50,hr-2+45,hr-2+40,hr-2+35,hr-2+30,hr-2+25,hr-2+20,hr-2+15,hr-2+10,hr-2+05,hr-2+00,hr-1+55,hr-1+50,hr-1+45,hr-1+40,hr-1+35,hr-1+30,hr-1+25,hr-1+20,hr-1+15,hr-1+10,hr-1+05,hr-1+00,hr-0+55,hr-0+50,hr-0+45,hr-0+40,hr-0+35,hr-0+30,hr-0+25,hr-0+20,hr-0+15,hr-0+10,hr-0+05,hr-0+00,steps-5+55,steps-5+50,steps-5+45,steps-5+40,steps-5+35,steps-5+30,steps-5+25,steps-5+20,steps-5+15,steps-5+10,steps-5+05,steps-5+00,steps-4+55,steps-4+50,steps-4+45,steps-4+40,steps-4+35,steps-4+30,steps-4+25,steps-4+20,steps-4+15,steps-4+10,steps-4+05,steps-4+00,steps-3+55,steps-3+50,steps-3+45,steps-3+40,steps-3+35,steps-3+30,steps-3+25,steps-3+20,steps-3+15,steps-3+10,steps-3+05,steps-3+00,steps-2+55,steps-2+50,steps-2+45,steps-2+40,steps-2+35,steps-2+30,steps-2+25,steps-2+20,steps-2+15,steps-2+10,steps-2+05,steps-2+00,steps-1+55,steps-1+50,steps-1+45,steps-1+40,steps-1+35,steps-1+30,steps-1+25,steps-1+20,steps-1+15,steps-1+10,steps-1+05,steps-1+00,steps-0+55,steps-0+50,steps-0+45,steps-0+40,steps-0+35,steps-0+30,steps-0+25,steps-0+20,steps-0+15,steps-0+10,steps-0+05,steps-0+00,cals-5+55,cals-5+50,cals-5+45,cals-5+40,cals-5+35,cals-5+30,cals-5+25,cals-5+20,cals-5+15,cals-5+10,cals-5+05,cals-5+00,cals-4+55,cals-4+50,cals-4+45,cals-4+40,cals-4+35,cals-4+30,cals-4+25,cals-4+20,cals-4+15,cals-4+10,cals-4+05,cals-4+00,cals-3+55,cals-3+50,cals-3+45,cals-3+40,cals-3+35,cals-3+30,cals-3+25,cals-3+20,cals-3+15,cals-3+10,cals-3+05,cals-3+00,cals-2+55,cals-2+50,cals-2+45,cals-2+40,cals-2+35,cals-2+30,cals-2+25,cals-2+20,cals-2+15,cals-2+10,cals-2+05,cals-2+00,cals-1+55,cals-1+50,cals-1+45,cals-1+40,cals-1+35,cals-1+30,cals-1+25,cals-1+20,cals-1+15,cals-1+10,cals-1+05,cals-1+00,cals-0+55,cals-0+50,cals-0+45,cals-0+40,cals-0+35,cals-0+30,cals-0+25,cals-0+20,cals-0+15,cals-0+10,cals-0+05,cals-0+00,bg+1+00,hour,minute,part_of_day,hour_sin,hour_cos,bg_sum_5hr,bg_mean_5hr,insulin_sum_5hr,insulin_mean_5hr,hr_sum_5hr,hr_mean_5hr,steps_sum_5hr,steps_mean_5hr,cals_sum_5hr,cals_mean_5hr
0,p01,8.2,13.8,9.6,8.2,13.4,9.7,8.2,12.8,9.2,8.2,15.5,8.7,11.1,14.8,8.4,10.3,12.7,8.1,10.1,11.4,8.3,10.1,11.9,9.6,10.2,15.1,11.1,10.3,17.1,11.8,10.2,17.9,12.8,9.9,17.4,13.9,9.9,16.1,14.2,9.8,14.3,14.2,9.3,12.

In [ ]:
%%time

if CONFIG.TRAIN_CATBOOST:
    catboost_models = train_catboost(train)

if CONFIG.TRAIN_XGB:
    xgboost_models = train_xgboost(train)


if CONFIG.TRAIN_LGB:
    lgb_models = train_lgb(train)

Training with seed: 42 for feature importance calculation...
Fold 1/5 with seed 42...
[0]	train-rmse:2.92733	valid-rmse:3.12122
[200]	train-rmse:1.71105	valid-rmse:2.22503
[360]	train-rmse:1.61437	valid-rmse:2.22760
Fold 1 RMSE: 2.2276
Fold 2/5 with seed 42...
[0]	train-rmse:2.92129	valid-rmse:3.12883
[200]	train-rmse:1.69612	valid-rmse:2.28839
[394]	train-rmse:1.57136	valid-rmse:2.31274
Fold 2 RMSE: 2.3127
Fold 3/5 with seed 42...
[0]	train-rmse:2.97273	valid-rmse:2.97118
[200]	train-rmse:1.79653	valid-rmse:1.92845
[330]	train-rmse:1.70019	valid-rmse:1.96726
Fold 3 RMSE: 1.9673
Fold 4/5 with seed 42...
[0]	train-rmse:2.89688	valid-rmse:3.20233


KeyboardInterrupt: 

In [ ]:


#lgbm 2.06
# std: 0.144

# xgboost
# STD of RMSE across folds: 0.1352
# OOF RMSE across seeds: 2.0997

In [ ]:
test = pd.read_csv('/content/data/brist-1d/test.csv')
test.head()

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,bg-5:15,bg-5:10,bg-5:05,bg-5:00,bg-4:55,bg-4:50,bg-4:45,bg-4:40,bg-4:35,bg-4:30,bg-4:25,bg-4:20,bg-4:15,bg-4:10,bg-4:05,bg-4:00,bg-3:55,bg-3:50,bg-3:45,bg-3:40,bg-3:35,bg-3:30,bg-3:25,bg-3:20,bg-3:15,bg-3:10,bg-3:05,bg-3:00,bg-2:55,bg-2:50,bg-2:45,bg-2:40,bg-2:35,bg-2:30,bg-2:25,bg-2:20,bg-2:15,bg-2:10,bg-2:05,bg-2:00,bg-1:55,bg-1:50,bg-1:45,bg-1:40,bg-1:35,bg-1:30,bg-1:25,bg-1:20,bg-1:15,bg-1:10,bg-1:05,bg-1:00,bg-0:55,bg-0:50,bg-0:45,bg-0:40,bg-0:35,bg-0:30,bg-0:25,bg-0:20,bg-0:15,bg-0:10,bg-0:05,bg-0:00,insulin-5:55,insulin-5:50,insulin-5:45,insulin-5:40,insulin-5:35,insulin-5:30,insulin-5:25,insulin-5:20,insulin-5:15,insulin-5:10,insulin-5:05,insulin-5:00,insulin-4:55,insulin-4:50,insulin-4:45,insulin-4:40,insulin-4:35,insulin-4:30,insulin-4:25,insulin-4:20,insulin-4:15,insulin-4:10,insulin-4:05,insulin-4:00,insulin-3:55,insulin-3:50,insulin-3:45,insulin-3:40,insulin-3:35,insulin-3:30,insulin-3:25,insulin-3:20,insulin-3:15,insulin-3:10,insulin-3:05,insulin-3:00,insulin-2:55,insulin-2:50,insulin-2:45,insulin-2:40,insulin-2:35,insulin-2:30,insulin-2:25,insulin-2:20,insulin-2:15,insulin-2:10,insulin-2:05,insulin-2:00,insulin-1:55,insulin-1:50,insulin-1:45,insulin-1:40,insulin-1:35,insulin-1:30,insulin-1:25,insulin-1:20,insulin-1:15,insulin-1:10,insulin-1:05,insulin-1:00,insulin-0:55,insulin-0:50,insulin-0:45,insulin-0:40,insulin-0:35,insulin-0:30,insulin-0:25,insulin-0:20,insulin-0:15,insulin-0:10,insulin-0:05,insulin-0:00,carbs-5:55,carbs-5:50,carbs-5:45,carbs-5:40,carbs-5:35,carbs-5:30,carbs-5:25,carbs-5:20,carbs-5:15,carbs-5:10,carbs-5:05,carbs-5:00,carbs-4:55,carbs-4:50,carbs-4:45,carbs-4:40,carbs-4:35,carbs-4:30,carbs-4:25,carbs-4:20,carbs-4:15,carbs-4:10,carbs-4:05,carbs-4:00,carbs-3:55,carbs-3:50,carbs-3:45,carbs-3:40,carbs-3:35,carbs-3:30,carbs-3:25,carbs-3:20,carbs-3:15,carbs-3:10,carbs-3:05,carbs-3:00,carbs-2:55,carbs-2:50,carbs-2:45,carbs-2:40,carbs-2:35,carbs-2:30,carbs-2:25,carbs-2:20,carbs-2:15,carbs-2:10,carbs-2:05,carbs-2:00,carbs-1:55,carbs-1:50,carbs-1:45,carbs-1:40,carbs-1:35,carbs-1:30,carbs-1:25,carbs-1:20,carbs-1:15,carbs-1:10,carbs-1:05,carbs-1:00,carbs-0:55,carbs-0:50,carbs-0:45,carbs-0:40,carbs-0:35,carbs-0:30,carbs-0:25,carbs-0:20,carbs-0:15,carbs-0:10,carbs-0:05,carbs-0:00,hr-5:55,hr-5:50,hr-5:45,hr-5:40,hr-5:35,hr-5:30,hr-5:25,hr-5:20,hr-5:15,hr-5:10,hr-5:05,hr-5:00,hr-4:55,hr-4:50,hr-4:45,hr-4:40,hr-4:35,hr-4:30,hr-4:25,hr-4:20,hr-4:15,hr-4:10,hr-4:05,hr-4:00,hr-3:55,hr-3:50,hr-3:45,hr-3:40,hr-3:35,hr-3:30,hr-3:25,hr-3:20,hr-3:15,hr-3:10,hr-3:05,hr-3:00,hr-2:55,hr-2:50,hr-2:45,hr-2:40,hr-2:35,hr-2:30,hr-2:25,hr-2:20,hr-2:15,hr-2:10,hr-2:05,hr-2:00,hr-1:55,hr-1:50,hr-1:45,hr-1:40,hr-1:35,hr-1:30,hr-1:25,hr-1:20,hr-1:15,hr-1:10,hr-1:05,hr-1:00,hr-0:55,hr-0:50,hr-0:45,hr-0:40,hr-0:35,hr-0:30,hr-0:25,hr-0:20,hr-0:15,hr-0:10,hr-0:05,hr-0:00,steps-5:55,steps-5:50,steps-5:45,steps-5:40,steps-5:35,steps-5:30,steps-5:25,steps-5:20,steps-5:15,steps-5:10,steps-5:05,steps-5:00,steps-4:55,steps-4:50,steps-4:45,steps-4:40,steps-4:35,steps-4:30,steps-4:25,steps-4:20,steps-4:15,steps-4:10,steps-4:05,steps-4:00,steps-3:55,steps-3:50,steps-3:45,steps-3:40,steps-3:35,steps-3:30,steps-3:25,steps-3:20,steps-3:15,steps-3:10,steps-3:05,steps-3:00,steps-2:55,steps-2:50,steps-2:45,steps-2:40,steps-2:35,steps-2:30,steps-2:25,steps-2:20,steps-2:15,steps-2:10,steps-2:05,steps-2:00,steps-1:55,steps-1:50,steps-1:45,steps-1:40,steps-1:35,steps-1:30,steps-1:25,steps-1:20,steps-1:15,steps-1:10,steps-1:05,steps-1:00,steps-0:55,steps-0:50,steps-0:45,steps-0:40,steps-0:35,steps-0:30,steps-0:25,steps-0:20,steps-0:15,steps-0:10,steps-0:05,steps-0:00,cals-5:55,cals-5:50,cals-5:45,cals-5:40,cals-5:35,cals-5:30,cals-5:25,cals-5:20,cals-5:15,cals-5:10,cals-5:05,cals-5:00,cals-4:55,cals-4:50,cals-4:45,cals-4:40,cals-4:35,cals-4:30,cals-4:25,cals-4:20,cals-4:15,cals-4:10,cals-4:05,cals-4:00,cals-3:55,cals-3:50,cals-3:45,cals-3:40,cals-3:35,cals-3:30,cals-3:25,cal

In [ ]:
def infer_xgboost(test_data, all_models):

    X_test = test_data.drop(CONFIG.GROUP_COL, axis=1)
    cat_cols = X_test.select_dtypes(include='category').columns

    all_test_preds = np.zeros(X_test.shape[0])

    for seed_models in all_models:
        seed_preds = np.zeros(X_test.shape[0])

        # Loop over models from different folds
        for model in seed_models:
            dtest = xgb.DMatrix(X_test, enable_categorical=True)
            fold_preds = model.predict(dtest)
            seed_preds += fold_preds / len(seed_models)  # Average predictions over the folds

        all_test_preds += seed_preds / len(all_models)  # Average predictions over the seeds

    return all_test_preds

In [ ]:
def infer_catboost(test_data, all_models):

    X_test = test_data.drop(CONFIG.GROUP_COL, axis=1)
    cat_cols = X_test.select_dtypes(include='category').columns

    all_test_preds = np.zeros(X_test.shape[0])

    for seed_models in all_models:
        seed_preds = np.zeros(X_test.shape[0])
        for model in seed_models:
            fold_preds = model.predict(X_test)
            seed_preds += fold_preds / len(seed_models)  # Average predictions over the folds
        all_test_preds += seed_preds / len(all_models)  # Average predictions over the seeds

    return all_test_preds

In [ ]:
def infer_lightgbm(test_data, all_models):

    X_test = test_data.drop(CONFIG.GROUP_COL, axis=1)
    cat_cols = X_test.select_dtypes(include='category').columns

    all_test_preds = np.zeros(X_test.shape[0])

    for seed_models in all_models:
        seed_preds = np.zeros(X_test.shape[0])
        for model in seed_models:
            fold_preds = model.predict(X_test)
            seed_preds += fold_preds / len(seed_models)  # Average predictions over the folds

        all_test_preds += seed_preds / len(all_models)  # Average predictions over the seeds

    return all_test_preds

In [ ]:
test = preprocess(test)

test.shape

<ipython-input-69-3db8d089fbaf>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'])
<ipython-input-69-3db8d089fbaf>:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill')


(3644, 366)

In [ ]:
xgb_predictions = infer_xgboost(test, xgboost_models)
cb_predictions = infer_catboost(test, catboost_models)
lgb_predictions = infer_lightgbm(test, lgb_models)

NameError: name 'catboost_models' is not defined

In [ ]:
sub = pd.read_csv('/content/data/brist-1d/sample_submission.csv')
sub['bg+1:00'] = xgb_predictions
sub.to_csv('xgb_submission.csv', index=False)
print(sub.head())

In [ ]:
sub = pd.read_csv('/content/data/brist-1d/sample_submission.csv')
sub['bg+1:00'] = cb_predictions
sub.to_csv('cb_submission.csv', index=False)
print(sub.head())

In [ ]:
sub = pd.read_csv('/content/data/brist-1d/sample_submission.csv')
sub['bg+1:00'] = lgb_predictions
sub.to_csv('lgb_submission.csv', index=False)
print(sub.head())